In [408]:
from PyPDF2 import PdfReader
import re

import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

from openai import OpenAI

from dotenv import load_dotenv

from tqdm.auto import tqdm
import json

import pandas as pd
import numpy as np



In [302]:
#!pip install pandas

# 

In [84]:
load_dotenv('.env')

True

# 

# 

# RAG Flow

In [26]:
def clean_text(text):
    # Cleaning special characters and normalizing the text

    clean_text = re.sub(r'\s+', ' ', text)  # Cleaning extra whitespaces
    #clean_text = re.sub(r'[^\w\s]', '', clean_text)  # Cleaning special characters
    clean_text = re.sub(r'14/9/24, 15:51 Lionel Messi - Wikipedia, la enciclopedia libre https://es\.wikipedia\.org/wiki/Lionel_Messi \d+/\d+', '', clean_text)   # Cleaning footer

    return clean_text


In [ ]:
chroma_client = chromadb.Client()

In [118]:
try:
     chroma_client.delete_collection(name="MessiDocuments") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible
except:
     pass

ValueError: Collection MessiDocument does not exist.

In [58]:
# Creating an instance of ChromaDB, the vector database used for storing the data

collection = chroma_client.create_collection(name="MessiDocuments", 
                                             metadata={"hnsw:space": "cosine"})

In [65]:
# Setting up the Sentence Transformer Model for embeddings
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\TALIGENT\Desktop\Proyectos\LLM-Zoomcamp\MessiXpert-RAG\messiXpert-RAG-venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [85]:
OpenAI_client = OpenAI()

In [27]:
# Reading the PDF 
reader = PdfReader('Lionel Messi - RAG Source.pdf')

num_pages = len(reader.pages)

print(f'Total Pages: {num_pages}')

Total Pages: 38


In [69]:
page_num = 0

for page in reader.pages:
    text = page.extract_text()
    cleaned_text = clean_text(text)

    #Embedding the text with sentence transformer
    embedding = embeddings_model.encode(cleaned_text).tolist()


    # Adding the document to the vector DB collection
    collection.add(
        documents=[cleaned_text],
        embeddings=[embedding],
        metadatas=[{
            "document_ID": "https://es.wikipedia.org/wiki/Lionel_Messi",
            "page_number": page_num + 1,
            "Fuente": "PDF",
            "Titulo": "Lionel_Messi_Wikipedia",
            "Autor": "Wikipedia Community"
        }],
        ids=[f"Lionel_Messi_Wikipedia_Page-{page_num + 1}"]
    )

    page_num+=1

    print(page_num)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [541]:
import requests
from bs4 import BeautifulSoup

def scrape_wikipedia_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraer el contenido principal del artículo
    content = soup.find('div', {'class': 'mw-parser-output'})
    
    # Extraer los párrafos y encabezados
    paragraphs = content.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'table'])
    
    return paragraphs

# Ejemplo de uso
url = "https://es.wikipedia.org/wiki/Lionel_Messi"
paragraphs = scrape_wikipedia_page(url)


In [496]:
def print_element(element):
    if element.name == 'table':
        table_html = str(element)
        table_df = pd.read_html(table_html)[0]
        print(table_df)
    else:
        print(element.text.strip())

In [563]:
def create_chunks_with_headers(elements):
    chunks = []
    current_headers = {f'h{i}': '' for i in range(1, 7)}

    for elem in elements:
        if elem.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            level = int(elem.name[1])
            current_headers[f'h{level}'] = elem.text.strip()
            # Limpiar los encabezados de nivel inferior
            for i in range(level + 1, 7):
                current_headers[f'h{i}'] = ''
        elif elem.name == 'p':
            paragraph = elem.text.strip()
            if paragraph:
                headers_concat = " > ".join([current_headers[f'h{i}'] for i in range(1, 7) if current_headers[f'h{i}']])
                chunks.append({'content': paragraph, 'headers_concat': headers_concat})
        elif elem.name == 'table':
            table_html = str(elem)
            try:
                table_df = pd.read_html(table_html)[0]
                table_text = table_df.to_string(index=False)
                headers_concat = " > ".join([current_headers[f'h{i}'] for i in range(1, 7) if current_headers[f'h{i}']])
                chunks.append({'content': table_text, 'headers_concat': headers_concat})
            except ValueError:
                headers_concat = " > ".join([current_headers[f'h{i}'] for i in range(1, 7) if current_headers[f'h{i}']])
                chunks.append({'content': 'Error al procesar la tabla', 'headers_concat': headers_concat})
    
    return chunks

In [564]:
# Ejemplo de uso
url = "https://es.wikipedia.org/wiki/Lionel_Messi"
elements = scrape_wikipedia_page(url)
chunks = create_chunks_with_headers(elements)

# Convertir a DataFrame
df = pd.DataFrame(chunks)
print(df.head())

                                             content headers_concat
0                                                ...               
1                                0               ...               
2  Lionel Andrés Messi Cuccittini (Rosario, 24 de...               
3  Con el Fútbol Club Barcelona, al que estuvo li...               
4  Considerado con frecuencia el mejor jugador de...               


C:\Users\TALIGENT\AppData\Local\Temp\ipykernel_18028\4201452077.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df = pd.read_html(table_html)[0]
C:\Users\TALIGENT\AppData\Local\Temp\ipykernel_18028\4201452077.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df = pd.read_html(table_html)[0]
C:\Users\TALIGENT\AppData\Local\Temp\ipykernel_18028\4201452077.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df = pd.read_html(table_html)[0]
C:\Users\TALIGENT\AppData\Local\Temp\ipykernel_18028\4201452077.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed i

In [555]:
df[:1]

,content,headers_concat
0,...,


In [565]:
# Recorrer e imprimir los primeros 10 registros del DataFrame
for index, row in df[10:60].iterrows():
    print(f"HEADER = {row['headers_concat']}")
    print(f"CONTENT = {row['content']}")
    print("__________________________________________________________")

HEADER = Orígenes y formación
CONTENT = Con apenas cuatro años, comenzó a practicar fútbol con Salvador Aparicio en el club Abanderado Grandoli, ubicado en el barrio Grandoli, a pocas cuadras de su casa.[13]​[16]​ En 1994, empezó a entrenarse en las divisiones inferiores de Newell's Old Boys.[13]​ En 1995, jugó un torneo no oficial con Central Córdoba.[17]​
__________________________________________________________
HEADER = Orígenes y formación
CONTENT = A la edad de ocho años, le fue diagnosticada una deficiencia de la hormona de crecimiento,[18]​ por lo que debió hacer un tratamiento de 900 dólares mensuales que, durante un año y medio, cubrieron su obra social y Acindar, siderúrgica en la que trabajaba su padre.[13]​
__________________________________________________________
HEADER = Orígenes y formación
CONTENT = En 1999, quiso ficharlo el equipo italiano Como, pero finalmente no lo hizo por las dificultades que representaba la mudanza de la familia.[19]​ Al año siguiente, dio su p

In [571]:
len(df)

325

In [573]:
len(df['headers_concat'].drop_duplicates())

71

In [575]:
tst = df.copy()

In [610]:
# Contar las apariciones del símbolo '>'
tst['conteo_mayor'] = tst['headers_concat'].str.count('>')

# Filtrar filas donde el conteo sea mayor que un valor específico
valor_maximo = 2
tst_filtrado = tst[tst['conteo_mayor'] <= valor_maximo]


len(tst_filtrado['headers_concat'].drop_duplicates())

37

In [538]:
h2 = content.find_all(['p'])

len(h2)

308

In [539]:
for elem in h2[:10]:  # Mostrar los primeros 10 elementos
    print_element(elem)

Lionel Andrés Messi Cuccittini (Rosario, 24 de junio de 1987), conocido como Leo Messi, es un futbolista argentino que juega como delantero o centrocampista. Desde 2023, integra el plantel del Inter Miami de la MLS canadoestadounidense. Es también internacional con la selección de Argentina, de la que es capitán.
Con el Fútbol Club Barcelona, al que estuvo ligado más de veinte años, ganó 35 títulos, entre ellos, diez de La Liga, cuatro de la Liga de Campeones de la UEFA y siete de la Copa del Rey.
Considerado con frecuencia el mejor jugador del mundo y uno de los mejores de todos los tiempos,[10]​ es el único en la historia que ha ganado, entre otras distinciones, ocho veces el Balón de Oro, ocho premios de la FIFA al mejor jugador del mundo, seis Botas de Oro y dos Balones de Oro de la Copa Mundial de Fútbol. En 2020, se convirtió en el primer futbolista y el primer argentino en recibir un premio Laureus y fue incluido en el Dream Team del Balón de Oro.
Tiene, entre otros, los récords

In [457]:
def queryDB_embeddings(query, collection):
    # Realizar una búsqueda de ejemplo
    results = collection.query(
        query_embeddings=embeddings_model.encode(query).tolist(),
        n_results=3
        )

    return results

In [149]:
def build_prompt(query, search_results_text_list):
    
    prompt_template = """
You are an expert biographer on the life and career of Lionel Messi, with deep knowledge of his entire history and statistics. 
Your task is to answer users questions based solely on the context provided to you. 
Answer respectfully and in a warm way, as if you are an assistant.
Answer in the same language that you are asked, if you are asked in english, answer in english, but if you are asked in spanish, answer in spanish.
Use only the data from the context to answer the question. 
If you cannot answer the question with the provided information, respond: “I’m sorry, but I don’t have enough information to answer that. Is there anything else I can help you with?”

CONTEXT: {context}

QUESTION: 
{question}
""".strip()

    context = ""
    
    for text in search_results_text_list:
        context = context + f"{text}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [185]:
def llm(prompt):
    response = OpenAI_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Printing the input tokens and output tokens of each all
    input_tokens = response.usage.prompt_tokens
    output_tokens = response.usage.completion_tokens

    input_tokens_cost_per_1k = 0.00015
    output_tokens_cost_per_1k = 0.0006

    input_tokens_cost = input_tokens_cost_per_1k * (input_tokens / 1000)
    output_tokens_cost = output_tokens_cost_per_1k * (output_tokens / 1000)
    total_cost = input_tokens_cost + output_tokens_cost

    print("------------------------------------")
    print(f"Input Tokens: {input_tokens}       Cost: ${input_tokens_cost:.8f}")
    print(f"Completion Tokens: {output_tokens}       Cost: ${output_tokens_cost:.8f}")
    print(f"Total Cost: ${total_cost:.8f}")
    print("------------------------------------")

    return response.choices[0].message.content

In [191]:
def generate_answer(question, collection):
    top_k_chunks = queryDB_embeddings(query=question, collection=collection)
    builded_prompt = build_prompt(query=question, search_results_text_list=top_k_chunks['documents'])
    answer = llm(builded_prompt)

    return answer


In [192]:
print(generate_answer("Cuantos hermanos tiene Messi?", collection))

------------------------------------
Input Tokens: 3882       Cost: $0.00058230
Completion Tokens: 43       Cost: $0.00002580
Total Cost: $0.00060810
------------------------------------
Lionel Messi tiene dos hermanos mayores, Rodrigo y Matías, y una hermana menor, María Sol. Por lo tanto, tiene un total de tres hermanos. ¿Hay algo más en lo que te pueda ayudar?


# 

# 

# Ground Truth Generation

In [446]:
def build_ground_truth_prompt(document_text):
    
    prompt_template = """
        You are an user of a chatbot assistant that is an expert biographer on the life and career of Lionel Messi,  
        with deep knowledge of his entire history and statistics.
        Your task is to generate five questions about the text context sent to you, to generate a ground truth dataset for the app.
        Please, formulate five questions that an user might ask based on the provided text.
        The recourd should contain the answer to the question, and the question should be complete an not too short. Use as fewer
        words as possible from the text provided.
        Generate the questions only in spanish.

        TEXT PROVIDED: 
        {document_text}

        Provide the output in parsable JSON without using code blocks:
        {{"questions": ["Question 1", "Question 2", "Question 3", "Question 4", "Question 5"]}}
    """.strip()
    
    prompt = prompt_template.format(document_text=document_text).strip()
    return prompt

In [447]:
def generate_questions(document):
    builded_prompt = build_ground_truth_prompt(document)
    answer = llm(builded_prompt)
    return answer


In [448]:
chunks = collection.get()

questions = {}

In [449]:
for doc_id, doc in tqdm(zip(chunks['ids'], chunks['documents'])):
    
    if doc_id in questions:
        continue

    generated_questions = generate_questions(doc)

    questions[doc_id] = json.loads(generated_questions)['questions']


1it [00:01,  1.81s/it]

------------------------------------
Input Tokens: 1352       Cost: $0.00020280
Completion Tokens: 123       Cost: $0.00007380
Total Cost: $0.00027660
------------------------------------


2it [00:02,  1.44s/it]

------------------------------------
Input Tokens: 1545       Cost: $0.00023175
Completion Tokens: 120       Cost: $0.00007200
Total Cost: $0.00030375
------------------------------------


3it [00:04,  1.50s/it]

...

------------------------------------
Input Tokens: 1473       Cost: $0.00022095
Completion Tokens: 106       Cost: $0.00006360
Total Cost: $0.00028455
------------------------------------


37it [00:47,  1.18s/it]

------------------------------------
Input Tokens: 1466       Cost: $0.00021990
Completion Tokens: 132       Cost: $0.00007920
Total Cost: $0.00029910
------------------------------------


38it [00:48,  1.28s/it]

------------------------------------
Input Tokens: 1464       Cost: $0.00021960
Completion Tokens: 142       Cost: $0.00008520
Total Cost: $0.00030480
------------------------------------


In [450]:
questions

{'Lionel_Messi_Wikipedia_Page-1': ['¿Cuál es el nombre completo de Lionel Messi y cuál es su apodo más conocido en el mundo del fútbol?',
  '¿En qué club profesional hizo su debut Lionel Messi y en qué fecha ocurrió?',
  '¿Cuántos Balones de Oro ha ganado Messi a lo largo de su carrera y cuáles fueron los años en los que los recibió?',
  '¿Qué logros alcanzó Messi con la selección argentina en su trayectoria internacional, incluyendo los torneos en los que participó?',
  '¿Cuáles son los clubes en los que ha jugado Lionel Messi desde su debut hasta la actualidad?'],
 'Lionel_Messi_Wikipedia_Page-10': ['¿Cuántos goles anotó Messi en 2012 para romper el récord de Gerd Müller?',
  '¿Qué logró Messi al marcar 91 goles en un año natural y cuántos de esos goles fueron con su club?',
  '¿Qué contrato renovó Messi en diciembre de2012 y cuál era su sueldo neto?',
  '¿Qué récord histórico alcanzó Messi al superar a Di Stéfano en goles de El Clásico?',
  '¿Cuántos goles anotó Messi en La Liga par

In [451]:
# Converting the questions into a question-row dataframe, to store as a csv
data = [(chunk_id, question) for chunk_id, questions in questions.items() for question in questions]

questions_df = pd.DataFrame(data, columns=['chunk_id', 'question'])

questions_df.to_csv('LionelMessiRAGSource-PageChunk-GroundTruth.csv', index=False, sep=';')

# 

# 

# Evaluating Retrieval

In [452]:
# Custom function to calculate the hit-rate of a vector search
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [453]:
# Custom function to calculate the MRR (Mean Reciprocal Rank) of a vector search
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [454]:
#Custom function to evaluate the search function on a ground truth dataset.
def evaluate(ground_truth, search_function):
    relevance_total = []

    for question in tqdm(ground_truth):
        chunk_id = question['chunk_id']
        results = search_function(question['question'])
        relevance = [id == chunk_id for id in results["ids"][0]]
        relevance_total.append(relevance)

    print(relevance_total)
    
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [455]:
questions_dict = questions_df.to_dict(orient='records')

In [458]:
evaluate(questions_dict, lambda query : queryDB_embeddings(query, collection))

100%|██████████| 190/190 [00:02<00:00, 66.23it/s]

[[True, False, False], [False, True, False], [False, False, False], [False, False, False], [False, True, False], [True, False, False], [False, True, False], [False, False, False], [False, False, True], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, True], [False, False, False], [True, False, False], [False, False, False], [True, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, True], [False, False, False], [False, False, False], [True, False, False], [False, False, False], [False, False, False], [False, False, False], [True, False, False], [True, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, True], [False, False, False], [False, True, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [T

{'hit_rate': 0.2894736842105263, 'mrr': 0.2131578947368421}

# 

# 

# Improving Metrics

Trying to improve metrics deleting stopwords and with lemmatizer (Specially for different verbs)

In [459]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TALIGENT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\TALIGENT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [460]:
def preprocess_query(query):
    # Convertir a minúsculas
    query = query.lower()
    
    # Tokenizar y eliminar stop words
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in query.split() if word not in stop_words]
    
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [461]:
evaluate(questions_dict, lambda query : queryDB_embeddings(preprocess_query(query), collection))

100%|██████████| 190/190 [00:03<00:00, 56.17it/s]

[[True, False, False], [True, False, False], [False, False, False], [False, False, False], [True, False, False], [True, False, False], [False, False, False], [False, False, False], [False, False, True], [False, True, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, True, False], [False, False, False], [False, False, True], [False, False, False], [True, False, False], [False, False, False], [False, False, False], [False, False, False], [False, True, False], [False, False, False], [False, False, False], [True, False, False], [False, False, False], [False, False, True], [False, False, False], [True, False, False], [True, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [

{'hit_rate': 0.2894736842105263, 'mrr': 0.21578947368421056}

# 

# 

# LLM AS A JUDGE

In [619]:
def build_llm_as_a_judge_prompt(question, llm_answer):
    
    prompt_template = """
        You are an expert evaluator for a RAG system.
        Your task is to analyze the relevance of the LLM generated answer to the users given question.
        The goal is to have 3 classification levels as if the relevance scores were 0, 0.5, or 1.
        Based on the relevance of the generated answer, you will classify it
        as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

        Here is the data for evaluation:

        Question: {question}
        Generated Answer: {llm_answer}

        Please analyze the content and context of the generated answer in relation to the question
        and provide your evaluation in parsable JSON without using code blocks:

        {{
        "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
        "Explanation": "[Provide a brief explanation for your evaluation in spanish]"
        }}
    """.strip()
    
    prompt = prompt_template.format(question=question, llm_answer=llm_answer).strip()
    return prompt

In [613]:
questions_dict

[{'chunk_id': 'Lionel_Messi_Wikipedia_Page-1',
  'question': '¿Cuál es el nombre completo de Lionel Messi y cuál es su apodo más conocido en el mundo del fútbol?'},
 {'chunk_id': 'Lionel_Messi_Wikipedia_Page-1',
  'question': '¿En qué club profesional hizo su debut Lionel Messi y en qué fecha ocurrió?'},
 {'chunk_id': 'Lionel_Messi_Wikipedia_Page-1',
  'question': '¿Cuántos Balones de Oro ha ganado Messi a lo largo de su carrera y cuáles fueron los años en los que los recibió?'},
 {'chunk_id': 'Lionel_Messi_Wikipedia_Page-1',
  'question': '¿Qué logros alcanzó Messi con la selección argentina en su trayectoria internacional, incluyendo los torneos en los que participó?'},
 {'chunk_id': 'Lionel_Messi_Wikipedia_Page-1',
  'question': '¿Cuáles son los clubes en los que ha jugado Lionel Messi desde su debut hasta la actualidad?'},
 {'chunk_id': 'Lionel_Messi_Wikipedia_Page-10',
  'question': '¿Cuántos goles anotó Messi en 2012 para romper el récord de Gerd Müller?'},
 {'chunk_id': 'Lionel_

In [634]:
evaluations = []

for record in tqdm(questions_dict):
    
    question = record['question']
    answer_llm = generate_answer(question, collection) 

    prompt = build_llm_as_a_judge_prompt(question, answer_llm)
    evaluation = llm(prompt)

    evaluation = json.loads(evaluation)
    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/190 [00:00<?, ?it/s]

------------------------------------
Input Tokens: 3781       Cost: $0.00056715
Completion Tokens: 29       Cost: $0.00001740
Total Cost: $0.00058455
------------------------------------


  1%|          | 1/190 [00:02<07:09,  2.27s/it]

------------------------------------
Input Tokens: 244       Cost: $0.00003660
Completion Tokens: 51       Cost: $0.00003060
Total Cost: $0.00006720
------------------------------------
You are an expert evaluator for a RAG system.
        Your task is to analyze the relevance of the LLM generated answer to the users given question.
        The goal is to have 3 classification levels as if the relevance scores were 0, 0.5, or 1.
        Based on the relevance of the generated answer, you will classify it
        as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

        Here is the data for evaluation:

        Question: ¿Cuál es el nombre completo de Lionel Messi y cuál es su apodo más conocido en el mundo del fútbol?
        Generated Answer: El nombre completo de Lionel Messi es Lionel Andrés Messi Cuccittini y su apodo más conocido en el mundo del fútbol es "Leo".

        Please analyze the content and context of the generated answer in relation to the question
        and prov

  1%|          | 2/190 [00:04<06:16,  2.00s/it]

...

------------------------------------
Input Tokens: 254       Cost: $0.00003810
Completion Tokens: 76       Cost: $0.00004560
Total Cost: $0.00008370
------------------------------------
You are an expert evaluator for a RAG system.
        Your task is to analyze the relevance of the LLM generated answer to the users given question.
        The goal is to have 3 classification levels as if the relevance scores were 0, 0.5, or 1.
        Based on the relevance of the generated answer, you will classify it
        as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

        Here is the data for evaluation:

        Question: ¿Cuál fue el premio que Messi recibió el 10 de enero de 2011 y cuántas veces lo había ganado hasta ese momento?
        Generated Answer: El 10 de enero de 2011, Messi recibió el Balón de Oro inaugural de la FIFA. Hasta ese momento, lo había ganado en tres ocasiones.

        Please analyze the content and context of the generated answer in relation to the question


 99%|█████████▉| 189/190 [07:03<00:02,  2.75s/it]

------------------------------------
Input Tokens: 236       Cost: $0.00003540
Completion Tokens: 76       Cost: $0.00004560
Total Cost: $0.00008100
------------------------------------
You are an expert evaluator for a RAG system.
        Your task is to analyze the relevance of the LLM generated answer to the users given question.
        The goal is to have 3 classification levels as if the relevance scores were 0, 0.5, or 1.
        Based on the relevance of the generated answer, you will classify it
        as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

        Here is the data for evaluation:

        Question: ¿Cuántos hat-tricks y pokers realizó Messi durante la temporada 2011-12?
        Generated Answer: Lo siento, pero no tengo suficiente información para responder a eso. ¿Hay algo más en lo que pueda ayudarte?

        Please analyze the content and context of the generated answer in relation to the question
        and provide your evaluation in parsable JSON withou

100%|██████████| 190/190 [07:06<00:00,  2.24s/it]

------------------------------------
Input Tokens: 287       Cost: $0.00004305
Completion Tokens: 65       Cost: $0.00003900
Total Cost: $0.00008205
------------------------------------
You are an expert evaluator for a RAG system.
        Your task is to analyze the relevance of the LLM generated answer to the users given question.
        The goal is to have 3 classification levels as if the relevance scores were 0, 0.5, or 1.
        Based on the relevance of the generated answer, you will classify it
        as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

        Here is the data for evaluation:

        Question: ¿Qué récord de goles en una temporada superó Messi durante la temporada 2011-12 y cuántos goles anotó en total?
        Generated Answer: Durante la temporada 2011-12, Messi superó el récord de más goles oficiales en una misma temporada (club y selección nacional) al anotar un total de 82 goles en 69 partidos. Este logro le permitió establecer un nuevo récord en la 

In [638]:
evaluations[0]

({'chunk_id': 'Lionel_Messi_Wikipedia_Page-1',
  'question': '¿Cuál es el nombre completo de Lionel Messi y cuál es su apodo más conocido en el mundo del fútbol?'},
 'El nombre completo de Lionel Messi es Lionel Andrés Messi Cuccittini y su apodo más conocido en el mundo del fútbol es "Leo".',
 {'Relevance': 'RELEVANT',
  'Explanation': 'La respuesta generada proporciona tanto el nombre completo de Lionel Messi como su apodo más conocido en el mundo del fútbol, cumpliendo completamente con la pregunta formulada.'})

In [645]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval = df_eval.reset_index()
df_eval.rename(columns={'index':'eval_id'}, inplace=True)

df_eval['chunk_id'] = df_eval.record.apply(lambda d: d['chunk_id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [648]:
df_eval = df_eval[['eval_id', 'chunk_id', 'question', 'answer', 'relevance', 'explanation']]

In [650]:
df_eval.relevance.value_counts()

relevance
NON_RELEVANT       91
RELEVANT           84
PARTLY_RELEVANT    15
Name: count, dtype: int64

In [651]:
df_eval.to_csv("LionelMessiRagSource-PageChunk-Evaluation.csv",sep=";")

In [653]:
""

''